In [4]:
import cv2
import numpy as np

def segment_blue_color(image):
    # Convert image to the HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define lower and upper bounds for blue color in HSV
    lower_blue = np.array([100, 50, 50])
    upper_blue = np.array([130, 255, 255])
    
    # Create a mask based on the blue color range
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    
    # Apply the mask to the original image
    segmented_image = cv2.bitwise_and(image, image, mask=mask)
    gray = cv2.cvtColor(segmented_image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (21, 21), 0)
    cv2.imshow('blurred Image', blurred)
    ret, thresh = cv2.threshold(blurred, 30, 255, 0)
    
    return thresh

cap = cv2.VideoCapture(0)
center_points = []  # List to store the center points of contours
append_counter = 0  # Counter for appending center points

while True:
    _, frame = cap.read()

    # Segment the image based on blue color
    segmented_image = segment_blue_color(frame)

    # Find contours
    contours, _ = cv2.findContours(segmented_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


    center = None

    if len(contours) > 0:
        c = max(contours, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        if M['m00'] != 0 :
            center = (int(M['m10']/M['m00']), int(M['m01']/M['m00']))

            if radius > 10:
                cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
                cv2.circle(frame, center, 5, (0, 0, 255), -1)

            center_points.append(center)

    # Draw the points from the center of the contour
    for point in center_points:
        if point != None :
            cv2.circle(frame, point, 10, (0, 0, 255), -1)

    # Increment the append counter
    append_counter += 1

    # Remove the oldest points if the list exceeds a certain length
    if len(center_points) > 30:
        del center_points[:15]

    # Display the original and segmented images
    cv2.imshow('Original Image', frame)
    cv2.imshow('Segmented Image', segmented_image)

    key = cv2.waitKey(1)
    # Exit the loop if 'q' is pressed
    if key == ord('q'):
        break

# Release the video capture and close the windows
cap.release()
cv2.destroyAllWindows()